In [1]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 5.0 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Masking, Bidirectional, GRU, TimeDistributed, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
# Download and unzip dataset
import gdown

url = 'https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu'
output = 'all_wav.zip'
gdown.download(url, output, quiet=False)
!unzip -q 'all_wav.zip' -d '/content/all_wav'

url = 'https://drive.google.com/uc?id=1vqvn0F0YYhEFbzLgP9wJ36vyInUnO5b5'
output = 'dataset.csv'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu
From (redirected): https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu&confirm=t&uuid=9b2b6827-796d-4a0b-80c8-a6e1a1dad0b1
To: /content/all_wav.zip
100%|██████████| 2.48G/2.48G [01:09<00:00, 35.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vqvn0F0YYhEFbzLgP9wJ36vyInUnO5b5
To: /content/dataset.csv
100%|██████████| 2.87M/2.87M [00:00<00:00, 134MB/s]


'dataset.csv'

In [4]:
df = pd.read_csv('/content/dataset.csv')
df.head()

,wav_filename,wav_filesize,transcript,confidence_level
0,./all_wav/Tehran_SayeRoshan0_101.wav,83044,اتفاقاتی که ندیده بودم,0.927557
1,./all_wav/Tehran_SayeRoshan0_105.wav,54468,مسجد,0.927557
2,./all_wav/Tehran_SayeRoshan0_107.wav,136036,جمع شدن مسلمین برای نمازهای جماعت,0.864152
3,./all_wav/Tehran_SayeRoshan0_108.wav,106788,همیشه برای محمدرضا پهلوی,0.927557
4,./all_wav/Tehran_SayeRoshan0_109.wav,170020,چه زمانی در کسوت شاه ایران نوکری اجانب را می‌کرد,0.854824


In [5]:
# Load dataset
df = pd.read_csv('/content/dataset.csv')

# Update paths for WAV files
df['wav_filename'] = df['wav_filename'].apply(lambda x: x.replace('./all_wav/', '/content/all_wav/all_wav/'))

# Filter dataset to include only existing WAV files
df = df[df['wav_filename'].apply(os.path.isfile)]

# Character map for Persian characters
char_map_str = """
' 0
<SPACE> 1
ا 2
ب 3
پ 4
ت 5
ث 6
ج 7
چ 8
ح 9
خ 10
د 11
ذ 12
ر 13
ز 14
ژ 15
س 16
ش 17
ص 18
ض 19
ط 20
ظ 21
ع 22
غ 23
ف 24
ق 25
ک 26
گ 27
ل 28
م 29
ن 30
و 31
ه 32
ی 33
، 34
؟ 35
"""
char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)] = ch
index_map[1] = ' '

# Ensure space character is in char_map
char_map[' '] = char_map['<SPACE>']

# Audio processing functions
def load_audio(file_path, sr=16000):
    audio, _ = librosa.load(file_path, sr=sr)
    return audio

def extract_features(audio, n_mfcc=13, sr=16000):
    mfcc_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfcc_features.T

# Prepare the dataset
X = []
y = []
input_lengths = []
label_lengths = []

for index, row in df.iterrows():
    audio_path = row['wav_filename']
    audio = load_audio(audio_path)
    features = extract_features(audio)
    X.append(features)
    input_lengths.append([features.shape[0]])
    label = [char_map.get(c, char_map[' ']) for c in row['transcript']]
    y.append(label)
    label_lengths.append([len(label)])

if len(X) == 0 or len(y) == 0:
    raise ValueError("No valid audio files were found. Please check the dataset and the paths.")

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', dtype='float32')
y = tf.keras.preprocessing.sequence.pad_sequences(y, padding='post', value=-1)

# Define the model
input_data = Input(name='the_input', shape=(None, 13))
masking_layer = Masking(mask_value=0.0)(input_data)
bilstm_layer_1 = Bidirectional(LSTM(128, return_sequences=True))(masking_layer)
bilstm_layer_2 = Bidirectional(LSTM(128, return_sequences=True))(bilstm_layer_1)
time_dense = TimeDistributed(Dense(len(char_map) + 1))(bilstm_layer_2)
y_pred = Activation('softmax', name='activation')(time_dense)

# Define the CTC loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Compile the model with CTC loss
labels = Input(name='the_labels', shape=[None], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss={'ctc': lambda y_true, y_pred: y_pred})

# Split the data into training and validation sets
X_train, X_val, y_train, y_val, input_length_train, input_length_val, label_length_train, label_length_val = train_test_split(X, y, input_lengths, label_lengths, test_size=0.2, random_state=42)

# Data generator
def data_generator(X, y, input_lengths, label_lengths, batch_size=32):
    while True:
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            y_batch = y[i:i+batch_size]
            input_lengths_batch = input_lengths[i:i+batch_size]
            label_lengths_batch = label_lengths[i:i+batch_size]
            yield (
                {
                    'the_input': np.array(X_batch),
                    'the_labels': np.array(y_batch),
                    'input_length': np.array(input_lengths_batch),
                    'label_length': np.array(label_lengths_batch)
                },
                np.zeros(len(X_batch))
            )

train_gen = data_generator(X_train, y_train, input_length_train, label_length_train, batch_size=32)
val_gen = data_generator(X_val, y_val, input_length_val, label_length_val, batch_size=32)

steps_per_epoch = len(X_train) // 32
validation_steps = len(X_val) // 32

# Train the model
model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=10, validation_data=val_gen, validation_steps=validation_steps)

# Save the model
model.save('asr_model.keras')

# Define inference model
inference_model = Model(inputs=input_data, outputs=y_pred)
inference_model.set_weights(model.get_weights()[:-2])

# Load a sample and predict
sample_index = 0
sample_features = X[sample_index]
sample_input_length = np.array([input_length_train[sample_index]])

sample_features = np.expand_dims(sample_features, axis=0)
sample_input_length = np.expand_dims(sample_input_length, axis=0)

preds = inference_model.predict(sample_features)
decoded_pred = tf.keras.backend.ctc_decode(preds, input_length=sample_input_length)[0][0]
decoded_pred = tf.keras.backend.get_value(decoded_pred)

predicted_text = ''.join([index_map[i] for i in decoded_pred if i != -1])

actual_text = df.iloc[sample_index]['transcript']

print(f"Predicted text: {predicted_text}")
print(f"Actual text: {actual_text}")

Epoch 1/10
609/609 [==============================] - 106s 143ms/step - loss: 100.7496 - val_loss: 86.7232
Epoch 2/10
609/609 [==============================] - 80s 132ms/step - loss: 81.3988 - val_loss: 76.7036
Epoch 3/10
609/609 [==============================] - 80s 132ms/step - loss: 74.8681 - val_loss: 72.2083
Epoch 4/10
609/609 [==============================] - 81s 133ms/step - loss: 70.9910 - val_loss: 69.4241
Epoch 5/10
609/609 [==============================] - 80s 131ms/step - loss: 68.2764 - val_loss: 67.6312
Epoch 6/10
609/609 [==============================] - 80s 132ms/step - loss: 66.0165 - val_loss: 66.2682
Epoch 7/10
609/609 [==============================] - 81s 133ms/step - loss: 64.2835 - val_loss: 65.7788
Epoch 8/10
609/609 [==============================] - 82s 134ms/step - loss: 62.7256 - val_loss: 64.9456
Epoch 9/10
609/609 [==============================] - 83s 136ms/step - loss: 61.4526 - val_loss: 63.7964
Epoch 10/10
609/609 [==============================] 

/usr/local/lib/python3.10/dist-packages/keras/src/saving/serialization_lib.py:403: UserWarning: The object being serialized includes a `lambda`. This is unsafe. In order to reload the object, you will have to pass `safe_mode=False` to the loading function. Please avoid using `lambda` in the future, and use named Python functions instead. This is the `lambda` being serialized: model.compile(optimizer=tf.keras.optimizers.Adam(), loss={'ctc': lambda y_true, y_pred: y_pred})

  return {key: serialize_keras_object(value) for key, value in obj.items()}


ValueError: You called `set_weights(weights)` on layer "model_1" with a weight list of length 12, but the layer was expecting 14 weights. Provided weights: [array([[-0.1562423 ,  0.04193013, -0.05970337, .....